In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import os
from os.path import join as oj
from sklearn import metrics
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import confusion_matrix
import numpy as np
from collections import Counter
from copy import deepcopy
from sklearn import metrics
plt.style.use('dark_background')
import pandas as pd
from colorama import Fore
import pickle as pkl
from tqdm import tqdm
import data, viz
from style import *
out_dir = 'results/jul22_19' # jun17_1

/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module

# compare all models

In [2]:
def load_results(out_dir):
    r = []
    for fname in tqdm(os.listdir(out_dir)):
        d = pkl.load(open(oj(out_dir, fname), 'rb'))
        d['param_name'] = fname[:-4]
        r.append(pd.Series(d))
    r = pd.concat(r, axis=1, sort=False).T.infer_objects()
    r = r.reindex(sorted(r.columns, reverse=True), axis=1) # sort the column names
    r = r.round(3)
    r = r.set_index('param_name')
    return r

results = load_results(out_dir)
# print(results.keys())

100%|██████████| 120/120 [00:00<00:00, 502.22it/s]


## look at prediction metrics

Most important thing is to have very low false negatives (very close to zero). After that, we also want to minimize the false positives.

In [3]:
# prev rule: sensitivity 0.97, specificity 0.425
r = results
# print(r.keys())
# r = r[[k for k in r if not 'std' in k]]
# r = r[[k for k in r if not '_test' in k]]
# r = r[[k for k in r if not '_f' in k]]
# r = r[r.index.str.contains('ros')] # only use random sampling
# r = r.sort_values(by=['balanced_accuracy'], ascending=False)

r = r[r['sensitivity_cv'] > 0.8] # what fraction of true pos do we return?
r = r[r['specificity_cv'] > 0.4] # what fraction of true negs do we return?

r = r.sort_values(by=['sensitivity_cv', 'specificity_cv'], ascending=False)
# r = r[['fn', 'tn', 'fp', 'tp', 'specificity', 'sensitivity', 'roc_auc', 'precision',
#        'f1', 'balanced_accuracy', 'accuracy']]
r = r[['sensitivity_cv', 'specificity_cv', # 'precision',
       'sensitivity_test1', 'specificity_test1',
       'sensitivity_test2', 'specificity_test2',
       'fn_cv', 'tn_cv', 'fp_cv', 'tp_cv',
       'precision_cv', 'roc_auc_cv', 
#        'f1_cv', 'balanced_accuracy_cv', 'accuracy_cv',
       'fn_test1', 'tn_test1', 'fp_test1', 'tp_test1',
       'fn_test2', 'tn_test2', 'fp_test2', 'tp_test2']]
# r.style.background_gradient(cmap='viridis', axis=None) # all values on same cmap
# r = r[r.index == 'logistic_select_lasso=27_ros=100']
# r.style.background_gradient(cmap='viridis', axis=0) # columns differently colored
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])

In [4]:
r

,sensitivity_cv,specificity_cv,sensitivity_test1,specificity_test1,sensitivity_test2,specificity_test2,fn_cv,tn_cv,fp_cv,tp_cv,precision_cv,roc_auc_cv,fn_test1,tn_test1,fp_test1,tp_test1,fn_test2,tn_test2,fp_test2,tp_test2
param_name,,,,,,,,,,,,,,,,,,,,
logistic_select_stab_lasso=6_sample_weights=500,0.969,0.522,0.873,0.514,0.984,0.509,4,4129,3774,126,0.032,0.898,37,8101,7651,255,4,4325,4179,244
logistic_select_stab_lasso=7_sample_weights=750,0.969,0.509,0.880,0.504,0.972,0.503,4,4026,3877,126,0.031,0.896,35,7943,7809,257,7,4278,4226,241
logistic_select_stab_lasso=6_sample_weights=750,0.969,0.504,0.887,0.500,0.984,0.493,4,3983,3920,126,0.031,0.897,33,7869,7883,259,4,4193,4311,244
logistic_select_stab_lasso=6_sample_weights=1000,0.969,0.504,0.887,0.500,0.984,0.493,4,3983,3920,126,0.031,0.897,33,7869,7883,259,4,4193,4311,244
logistic_select_stab_lasso=7_sample_weights=500,0.962,0.546,0.866,0.535,0.968,0.525,5,4312,3591,125,0.034,0.897,39,8423,7329,253,8,4463,4041,240
logistic_select_stab_lasso=5_sample_weights=500,0.954,0.531,0.863,0.524,0.984,0.516,6,4193,3710,124,0.032,0.884,40,8251,7501,252,4,4388,4116,244
logistic_select_stab_lasso=10_sample_weights=1000,0.954,0.434,0.897,0.424,0.976,0.386,6,3428,4475,124,0.027,0.888,30,6682,9070,262,6,3281,5223,242
logistic_select_stab_lasso=10_sample_weights=750,0.946,0.492,0.887,0.483,0.972,0.445,7,3886,4017,123,0.030,0.890,33,7605,8147,259,7,3788,4716,241
logistic_select_stab_lasso=10_sample_weights=500,0.938,0.557,0.866,0.550,0.968,0.509,8,4399,3504,122,0.034,0.893,39,8657,7095,253,8,4330,4174,240


**look at feat importances**

In [ ]:
r = results
best_model = 'logistic_select_rf=10_sample_weights=1000' #logistic_select_rf=5_smote=5'
# print(r.keys())
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])
r = r.sort_values('balanced_accuracy', ascending=False)
keys = [k for k in r if '_f' in k]
keys_remapped = {k: k.replace('_f', '') for k in keys}
r = r[keys].rename(columns=keys_remapped)
# r = r.sort_values('lifetime')
# r = r[r.index.str.contains('35')]
# r = r[r.index.str.contains('11')]
# r = r.rename(columns={'mean_square_displacement': 'msd', 'total_displacement': 'td'})

# r = r[r.index.str.contains('20')]
r = r[r.index.str.contains(best_model)]

r = r[[k for k in r if not 'std' in k]]


def rank(r):
    '''Rank feature importances appropriately
    '''
    r = r.abs()
    r = r.rank(axis=1, ascending=False, method='min')
    return r
# 
# r = rank(r)
r = r.reindex(r.mean().sort_values(ascending=True).index, axis=1) # sort cols by mean rank
idxs = r.index
# r.insert(0, 'balanced_acc', results.loc[idxs]['balanced_accuracy'])
# r = r.sort_values('balanced_acc', ascending=False)
# subset = list(r.keys())
# subset.remove('balanced_acc')
# r.to_html('table.html')
# r = r.fillna(0).style.background_gradient(cmap='viridis_r', axis=1) #, subset=subset) # rows differently colored


r = r.transpose()
r = r.reindex(r[best_model].abs().sort_values(ascending=False).index) # sort by abs value
# r = r.sort_values(by='logistic_20_ros=1')
r = r.fillna(0).style.background_gradient(cmap=cm) #, axis=0) #, subset=subset) # rows differently colored
# r = background_gradient(r)
r

# misc analysis

In [ ]:
def count_common_feats(m_cv, model_type='tree'):
    if model_type == 'tree':
        feats_used = []
        for i in range(len(m_cv['estimator'])):
            m_fit = m_cv['estimator'][i]
            feats_used += list(ks[m_fit.feature_importances_ != 0])
    elif model_type == 'logistic':
        feats_used = []
        for i in range(len(m_cv['estimator'])):
            m_fit = m_cv['estimator'][i]
            num_feats = 5
            # get top num_feats features with biggest bsolute weights
            idxs = np.abs(m_fit.coef_).flatten().argsort()[-num_feats:][::-1]
            feats_used += list(ks[idxs])
    return sorted(dict(Counter(feats_used)).items(), key=lambda kv: kv[1], reverse=True)
        
count_common_feats(m_cv, model_type)

In [ ]:
# plot a tree
m_fit = m_cv['estimator'][0]
plt.figure(dpi=300)
plot_tree(m_fit, feature_names=ks)
plt.show()